# Mod 4 Project - Starter Notebook

This notebook has been provided to you so that you can make use of the following starter code to help with the trickier parts of preprocessing the Zillow dataset. 

The notebook contains a rough outline the general order you'll likely want to take in this project. You'll notice that most of the areas are left blank. This is so that it's more obvious exactly when you should make use of the starter code provided for preprocessing. 

**_NOTE:_** The number of empty cells are not meant to infer how much or how little code should be involved in any given step--we've just provided a few for your convenience. Add, delete, and change things around in this notebook as needed!

# Some Notes Before Starting

This project will be one of the more challenging projects you complete in this program. This is because working with Time Series data is a bit different than working with regular datasets. In order to make this a bit less frustrating and help you understand what you need to do (and when you need to do it), we'll quickly review the dataset formats that you'll encounter in this project. 

## Wide Format vs Long Format

If you take a look at the format of the data in `zillow_data.csv`, you'll notice that the actual Time Series values are stored as separate columns. Here's a sample: 

<img src='~/../images/df_head.png'>

You'll notice that the first seven columns look like any other dataset you're used to working with. However, column 8 refers to the median housing sales values for April 1996, column 9 for May 1996, and so on. This This is called **_Wide Format_**, and it makes the dataframe intuitive and easy to read. However, there are problems with this format when it comes to actually learning from the data, because the data only makes sense if you know the name of the column that the data can be found it. Since column names are metadata, our algorithms will miss out on what dates each value is for. This means that before we pass this data to our ARIMA model, we'll need to reshape our dataset to **_Long Format_**. Reshaped into long format, the dataframe above would now look like:

<img src='~/../images/melted1.png'>

There are now many more rows in this dataset--one for each unique time and zipcode combination in the data! Once our dataset is in this format, we'll be able to train an ARIMA model on it. The method used to convert from Wide to Long is `pd.melt()`, and it is common to refer to our dataset as 'melted' after the transition to denote that it is in long format. 

# Helper Functions Provided

Melting a dataset can be tricky if you've never done it before, so you'll see that we have provided a sample function, `melt_data()`, to help you with this step below. Also provided is:

* `get_datetimes()`, a function to deal with converting the column values for datetimes as a pandas series of datetime objects
* Some good parameters for matplotlib to help make your visualizations more readable. 

Good luck!


# Step 1: Load the Data/Filtering for Chosen Zipcodes

In [1]:
import pandas as pd
import matplotlib

In [2]:
df = pd.read_csv('zillow_data.csv')

FileNotFoundError: [Errno 2] File b'zillow_data.csv' does not exist: b'zillow_data.csv'

In [ ]:
df.head()

In [ ]:
df['RegionName'].value_counts().unique()

# Step 2: Data Preprocessing

In [ ]:
def get_datetimes(df, first_date_col):
    """
    Converts year-month column headings to date-time.
    
    Args: 
        df: a dataframe with column headdings that are formatted yyyy-mm
        All columns follwing first_date_col should have same format
        first_date_col: the index of the first date column or first column 
        you want converted.
    Returns:
        Series of datetime64 dtype
    """
    return pd.to_datetime(df.columns.values[(first_date_col):], format='%Y-%m')

In [ ]:
datetimes = get_datetimes(df, 7)

In [ ]:
df.info()

# Step 3: EDA and Visualization

In [ ]:
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 22}

matplotlib.rc('font', **font)

# NOTE: if you visualizations are too cluttered to read, try calling 'plt.gcf().autofmt_xdate()'!

In [ ]:
data = df.loc[df['City'] == 'Chicago', '1996-04':]

In [ ]:
data.head(100)

In [ ]:
plt.plot(data.head(10))
plt.show();

# Step 4: Reshape from Wide to Long Format

In [ ]:
def melt_data(df):
    melted = pd.melt(df, id_vars=['RegionName', 'RegionID', 'City', 'State', 'Metro', 'CountyName', 'SizeRank'], var_name='time')
    melted['time'] = pd.to_datetime(melted['time'], infer_datetime_format=True)
    melted = melted.dropna(subset=['value'])
    return melted.groupby('time').aggregate({'value':'mean'})

In [ ]:
melted_df = melt_data(df)

In [ ]:
melted_df.head()

In [ ]:
chicago_melted = melt_data(df.loc[df['RegionName'] == 60657])

In [ ]:
plt.plot(chicago_melted)
plt.show()

# Step 5: ARIMA Modeling

In [ ]:
import seaborn as sns
%matplotlib inline
from matplotlib.pylab import rcParams
plt.style.use('fivethirtyeight')
from fbprophet import Prophet as proph

In [ ]:
def model_price(df, start, end, periods=36, freq='MS', interval_width=0.80):
    df = df.reset_index()
    df.columns = ['ds', 'y']
    data = df.loc[(df['ds'] > start) & (df['ds'] < end)]
    print(data)
    Model = proph(interval_width=interval_width)
    Model.fit(data)
    future_dates = Model.make_future_dataframe(periods=periods, freq=freq)
    forecast = Model.predict(future_dates)
    return Model

In [ ]:
model = model_price(chicago_melted, '1996-04-01', '2017-11-01')
Model.plot(forecast, uncertainty=True)
plt.show()

# Step 6: Interpreting Results